In [0]:
!pip install scikit-video

import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization
import matplotlib.pyplot as plt
import numpy as np

**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

(you can use Markdown and Latex)

It allows us to give a proper balance between exploitation and exploration ( we explore with probability $\epsilon$ ).

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))
        
        pos = self.position

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        
        self.t = 0

        #print('board')
        #print(self.board)
        #print('pos')
        #print(self.position)
        #print(self.grid_size)
        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        #print('big state')
        #print(state)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=20
temperature=0.3
epochs_train=50 # set small when debugging
epochs_test=5 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The position matrix allows us to know the position of our agent at each step in time.

The board matrix keeps track of where the bonus and malus are.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
      return np.random.randint(4)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)
            #print(state)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

            # Save as a mp4
            env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))
    

In [0]:
# Initialize the game
env = Environment(grid_size=size, max_time=20,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,20,prefix='random')
HTML(display_videos('random0.mp4'))

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




**Answer**

Using the definition of $Q^{\pi}(s,a)$:
\begin{equation*}
Q^{\pi}(s,a)=\sum_{s'}\mathbb{P}(s'|s,a)[r(s,a) + \gamma\sum_{a'}\pi(s',a')Q^{\pi}(s',a')] \\
= \mathbb{E}_{s' \sim p(.|s,a)}[r(s,a) + \gamma\mathbb{E}_{a' \sim \pi(s')}Q^{\pi}(s',a')] \\
= \mathbb{E}_{s' \sim p(.|s,a)}\mathbb{E}_{a' \sim \pi(s')}[r(s,a) + \gamma Q^{\pi}(s',a')] \\
= \mathbb{E}_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma 
Q^{\pi}(s',a')]
\end{equation*}

The optimal policy is the one that chosen the set of actions that maximizes $Q^{\pi}(s,a)$, it follows directly that :

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}

Because we want to approximate $Q^{*}(s,a)$, a good loss would be:

\begin{equation*}
\mathcal{L}(\theta)= (Q(s,a,\theta) - Q^{*}(s,a))^2.
\end{equation*}

Because we don't have access to $Q^{*}(s,a)$ directly, we can approximate it by $E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)$, therefore resulting in the following loss:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}



***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
      self.max_memory = max_memory
      self.memory = list()
      self.mem_size = 0

    def remember(self, m):
      self.memory.append(m)
      self.mem_size += 1
      if self.mem_size > self.max_memory:
          self.memory = self.memory[1:]
          self.mem_size -= 1

    def random_access(self):
      return self.memory[np.random.randint(self.mem_size)]

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0
    mean_score = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose
        
        #print(e, epoch, loss, win, lose, win-lose)

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        best_action = np.argmax(self.model.predict(np.array([s])))
        return best_action
      
    def get_Q(self):
        
        s = 0
        for state in possible_states:
          t = 0
          for a in actions:
            t += proba(state,a)*1

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        batch_size = min(self.batch_size,self.memory.mem_size)
        input_states = np.zeros((batch_size, 5,5,self.n_state))
        target_q = np.zeros((batch_size, 4))
        
        for i in range(batch_size):
            
            s, n_s, a, r, game_over = self.memory.random_access()
            input_states[i] = np.array(s)
            
            if game_over_:
              target_q[i,:][a] = r
            else:
              Q_pred = self.model.predict(np.array([n_s]))
              target_q[i,:][a] = r + self.discount*np.max(Q_pred)

        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)
        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        from keras.models import Model
        from keras.layers import Input, Dense
          
        #from keras.models import Sequential
        model = Sequential()

        from keras.layers import Dense, Activation, Flatten
        nb_neurons = 200
        model.add(Flatten(input_shape=(5,5,2)))
        model.add(Dense(nb_neurons))
        model.add(Activation('relu'))
        model.add(Dense(nb_neurons))
        model.add(Activation('relu'))
        model.add(Dense(4))  
        
        from keras.optimizers import Adam
        adam = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.00001, amsgrad=False)
        model.compile(loss='mse', optimizer='adam')
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.001), "mse")
        print('model compiled')
        self.model = model
        

In [36]:
# parameters
size = 13
T=100
temperature=0.3
epochs_train=100 # set small when debugging
epochs_test=50 # set small when debugging

env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.3, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train90.mp4'))

model compiled
Epoch 000/100 | Loss 0.0053 | Win/lose count 2.5/2.0 (0.5)
Epoch 001/100 | Loss 0.0134 | Win/lose count 3.0/6.0 (-3.0)
Epoch 002/100 | Loss 0.0343 | Win/lose count 5.5/4.0 (1.5)
Epoch 003/100 | Loss 0.0069 | Win/lose count 5.0/5.0 (0.0)
Epoch 004/100 | Loss 0.0203 | Win/lose count 4.5/3.0 (1.5)
Epoch 005/100 | Loss 0.0195 | Win/lose count 6.0/8.0 (-2.0)
Epoch 006/100 | Loss 0.0112 | Win/lose count 5.0/4.0 (1.0)
Epoch 007/100 | Loss 0.0124 | Win/lose count 7.0/2.0 (5.0)
Epoch 008/100 | Loss 0.0231 | Win/lose count 8.5/6.0 (2.5)
Epoch 009/100 | Loss 0.0132 | Win/lose count 7.0/3.0 (4.0)
Epoch 010/100 | Loss 0.0304 | Win/lose count 2.0/1.0 (1.0)
Epoch 011/100 | Loss 0.0188 | Win/lose count 6.5/3.0 (3.5)
Epoch 012/100 | Loss 0.0190 | Win/lose count 6.0/2.0 (4.0)
Epoch 013/100 | Loss 0.0092 | Win/lose count 6.0/5.0 (1.0)
Epoch 014/100 | Loss 0.0133 | Win/lose count 7.0/4.0 (3.0)
Epoch 015/100 | Loss 0.0267 | Win/lose count 3.0/2.0 (1.0)
Epoch 016/100 | Loss 0.0104 | Win/lose 

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        from keras.models import Sequential
        from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization

        ##############################
        model = Sequential()

        #layer1
        model.add( Conv2D(16, (2, 2), activation='relu') )
        #model.add(BatchNormalization())
        
        #layer2
        model.add( Conv2D(8, (2, 2), activation='relu' ) )
        #model.add(BatchNormalization())

        #flatten
        model.add(Flatten())
        model.add(Dense(256, activation='relu'))
        model.add(Activation('relu'))
        model.add(Dense(4))

        #Optimizer choice
        from keras.optimizers import Adam
        adam = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.00001, amsgrad=False)
        model.compile(loss='mse', optimizer=adam)
        
        #model.compile(sgd(lr=lr, decay=1e-4, momentum=0.001), "mse")
        
        self.model = model

In [27]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.2, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/100 | Loss 0.0216 | Win/lose count 6.5/6.0 (0.5)
Epoch 001/100 | Loss 0.0060 | Win/lose count 1.5/0 (1.5)
Epoch 002/100 | Loss 0.0187 | Win/lose count 9.0/8.0 (1.0)
Epoch 003/100 | Loss 0.0143 | Win/lose count 8.0/5.0 (3.0)
Epoch 004/100 | Loss 0.0281 | Win/lose count 2.0/3.0 (-1.0)
Epoch 005/100 | Loss 0.0109 | Win/lose count 5.0/5.0 (0.0)
Epoch 006/100 | Loss 0.0116 | Win/lose count 10.0/2.0 (8.0)
Epoch 007/100 | Loss 0.0194 | Win/lose count 2.0/2.0 (0.0)
Epoch 008/100 | Loss 0.0186 | Win/lose count 6.5/3.0 (3.5)
Epoch 009/100 | Loss 0.0068 | Win/lose count 6.0/1.0 (5.0)
Epoch 010/100 | Loss 0.0240 | Win/lose count 7.5/5.0 (2.5)
Epoch 011/100 | Loss 0.0104 | Win/lose count 4.0/6.0 (-2.0)
Epoch 012/100 | Loss 0.0134 | Win/lose count 3.5/1.0 (2.5)
Epoch 013/100 | Loss 0.0084 | Win/lose count 4.5/3.0 (1.5)
Epoch 014/100 | Loss 0.0093 | Win/lose count 7.0/0 (7.0)
Epoch 015/100 | Loss 0.0085 | Win/lose count 6.0/4.0 (2.0)
Epoch 016/100 | Loss 0.0093 | Win/lose count 4.0/2.0 (2.0

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [38]:
epochs_test = 5

env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.15, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.15, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

model compiled
Test of the CNN
Win/lose count 2.5/2.0. Average score (0.5)
Win/lose count 14.0/3.0. Average score (5.75)
Win/lose count 10.0/0. Average score (7.166666666666667)
Win/lose count 19.0/2.0. Average score (9.625)
Win/lose count 11.0/3.0. Average score (9.3)
Final score: 9.3
Test of the FC
Win/lose count 11.0/2.0. Average score (9.0)
Win/lose count 10.5/2.0. Average score (8.75)
Win/lose count 9.5/3.0. Average score (8.0)
Win/lose count 14.0/1.0. Average score (9.25)
Win/lose count 1.0/0. Average score (7.6)
Final score: 7.6


In [41]:
HTML(display_videos('cnn_test4.mp4'))

In [40]:
HTML(display_videos('fc_test4.mp4'))

**Answer**

Sometimes the algorithm will get stuck in the same spot or move forwards and backwards.
In particular, the algorithm using the fully connected network can sometimes get stuck because it isn't considering the spacial coherence of its input (it is viewing the board as one flattened vector).

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))
        
        pos = self.position

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        if train:
            reward -= self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        
        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state
      
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
          
            #update exploration/exploitation criterium
            agent.set_epsilon(agent.epsilon*0.9)
          
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

In [44]:
epochs_train = 100
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.15, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore90.mp4'))

Epoch 000/100 | Loss 0.0222 | Win/lose count 3.0/11.299999999999983 (-8.299999999999983)
Epoch 001/100 | Loss 0.0212 | Win/lose count 4.0/13.399999999999979 (-9.399999999999979)
Epoch 002/100 | Loss 0.0069 | Win/lose count 5.0/10.599999999999978 (-5.599999999999978)
Epoch 003/100 | Loss 0.0073 | Win/lose count 3.0/10.099999999999985 (-7.099999999999985)
Epoch 004/100 | Loss 0.0230 | Win/lose count 5.0/10.699999999999978 (-5.699999999999978)
Epoch 005/100 | Loss 0.0056 | Win/lose count 5.0/7.699999999999989 (-2.6999999999999886)
Epoch 006/100 | Loss 0.0092 | Win/lose count 6.0/7.999999999999988 (-1.9999999999999876)
Epoch 007/100 | Loss 0.0111 | Win/lose count 7.0/11.999999999999986 (-4.999999999999986)
Epoch 008/100 | Loss 0.0155 | Win/lose count 5.5/7.899999999999988 (-2.399999999999988)
Epoch 009/100 | Loss 0.0053 | Win/lose count 6.5/9.099999999999984 (-2.5999999999999837)
Epoch 010/100 | Loss 0.0068 | Win/lose count 8.0/6.8999999999999915 (1.1000000000000085)
Epoch 011/100 | Loss 0

In [45]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore4.mp4'))

Win/lose count 15.5/3.0. Average score (12.5)
Win/lose count 19.5/8.0. Average score (12.0)
Win/lose count 12.5/3.0. Average score (11.166666666666666)
Win/lose count 22.5/4.0. Average score (13.0)
Win/lose count 16.0/3.0. Average score (13.0)
Final score: 13.0


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***